<a href="https://colab.research.google.com/github/FelixWag/DecisionTreeAndRandomForest/blob/master/Exercise3BFelixWagner01429339.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Mount Google Drive**

In [16]:
# Mount my Google Drive
# Load the Drive helper and mount
import os
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

# Create a models folder to store the trained models in it
if not os.path.exists('models'):
    os.makedirs('models')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Download the Dataset and store it**

In [17]:
! wget -O development.1.zip "https://zenodo.org/record/45739/files/TUT-acoustic-scenes-2016-development.audio.1.zip?download=1"
! wget -O development.2.zip "https://zenodo.org/record/45739/files/TUT-acoustic-scenes-2016-development.audio.2.zip?download=1"
! wget -O development.3.zip "https://zenodo.org/record/45739/files/TUT-acoustic-scenes-2016-development.audio.3.zip?download=1"
! wget -O development.4.zip "https://zenodo.org/record/45739/files/TUT-acoustic-scenes-2016-development.audio.4.zip?download=1"
! wget -O development.5.zip "https://zenodo.org/record/45739/files/TUT-acoustic-scenes-2016-development.audio.5.zip?download=1"
! wget -O development.6.zip "https://zenodo.org/record/45739/files/TUT-acoustic-scenes-2016-development.audio.6.zip?download=1"
! wget -O development.7.zip "https://zenodo.org/record/45739/files/TUT-acoustic-scenes-2016-development.audio.7.zip?download=1"
! wget -O development.8.zip "https://zenodo.org/record/45739/files/TUT-acoustic-scenes-2016-development.audio.8.zip?download=1"

#! unzip development.1.zip -d drive/My\ Drive/TU/Music/PartB/Data/Development/
#! unzip development.2.zip -d drive/My\ Drive/TU/Music/PartB/Data/Development/
#! unzip development.3.zip -d drive/My\ Drive/TU/Music/PartB/Data/Development/
#! unzip development.4.zip -d drive/My\ Drive/TU/Music/PartB/Data/Development/
#! unzip development.5.zip -d drive/My\ Drive/TU/Music/PartB/Data/Development/
#! unzip development.6.zip -d drive/My\ Drive/TU/Music/PartB/Data/Development/
#! unzip development.7.zip -d drive/My\ Drive/TU/Music/PartB/Data/Development/
#! unzip development.8.zip -d drive/My\ Drive/TU/Music/PartB/Data/Development/
! unzip development.1.zip -d data_e/
! unzip development.2.zip -d data_e/
! unzip development.3.zip -d data_e/
! unzip development.4.zip -d data_e/
! unzip development.5.zip -d data_e/
! unzip development.6.zip -d data_e/
! unzip development.7.zip -d data_e/
! unzip development.8.zip -d data_e/

--2020-03-13 20:27:15--  https://zenodo.org/record/45739/files/TUT-acoustic-scenes-2016-development.audio.1.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.95.95
Connecting to zenodo.org (zenodo.org)|188.184.95.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1070981236 (1021M) [application/octet-stream]
Saving to: ‘development.1.zip’

development.1.zip   100%[===================>]   1021M  44.4MB/s    in 22s     

2020-03-13 20:27:38 (46.2 MB/s) - ‘development.1.zip’ saved [1070981236/1070981236]

--2020-03-13 20:27:39--  https://zenodo.org/record/45739/files/TUT-acoustic-scenes-2016-development.audio.2.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.95.95
Connecting to zenodo.org (zenodo.org)|188.184.95.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1067186166 (1018M) [application/octet-stream]
Saving to: ‘development.2.zip’

development.2.zip   100%[===================>]   1018M  48.8MB/s    in 18s    

In [0]:
# Now the evaluation set
! wget -O evaluation.1.zip "https://zenodo.org/record/165995/files/TUT-acoustic-scenes-2016-evaluation.audio.1.zip?download=1"
! wget -O evaluation.2.zip "https://zenodo.org/record/165995/files/TUT-acoustic-scenes-2016-evaluation.audio.2.zip?download=1"
! wget -O evaluation.3.zip "https://zenodo.org/record/165995/files/TUT-acoustic-scenes-2016-evaluation.audio.3.zip?download=1"
! unzip evaluation.1.zip -d drive/My\ Drive/TU/Music/PartB/Data/Evaluation/
! unzip evaluation.2.zip -d drive/My\ Drive/TU/Music/PartB/Data/Evaluation/
! unzip evaluation.3.zip -d drive/My\ Drive/TU/Music/PartB/Data/Evaluation/

In [18]:
! pip install numpy scipy torch matplotlib sklearn torchaudio --upgrade

Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.18.1)
Requirement already up-to-date: scipy in /usr/local/lib/python3.6/dist-packages (1.4.1)
Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.4.0)
Requirement already up-to-date: matplotlib in /usr/local/lib/python3.6/dist-packages (3.2.0)
Requirement already up-to-date: sklearn in /usr/local/lib/python3.6/dist-packages (0.0)
Requirement already up-to-date: torchaudio in /usr/local/lib/python3.6/dist-packages (0.4.0)


In [0]:
# All the necessary imports to do
import pandas as pd
import numpy as np
import time

import torchaudio
from torchaudio import transforms
import torch

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as torch_func
import torch.optim as optim
from torch.utils.data import Dataset as Dataset
from torch.autograd import Variable

In [0]:
# Read in the Data
# Paths to the respective files
DEVELOPMENT_METADATA = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/meta.txt"

FOLD1_TRAIN = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/evaluation_setup/fold1_train.txt"
FOLD1_TEST = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/evaluation_setup/fold1_test.txt"
FOLD1_EVALUATE = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/evaluation_setup/fold1_evaluate.txt"

FOLD2_TRAIN = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/evaluation_setup/fold2_train.txt"
FOLD2_TEST = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/evaluation_setup/fold2_test.txt"
FOLD2_EVALUATE = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/evaluation_setup/fold2_evaluate.txt"

FOLD3_TRAIN = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/evaluation_setup/fold3_train.txt"
FOLD3_TEST = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/evaluation_setup/fold3_test.txt"
FOLD3_EVALUATE = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/evaluation_setup/fold3_evaluate.txt"

FOLD4_TRAIN = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/evaluation_setup/fold4_train.txt"
FOLD4_TEST = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/evaluation_setup/fold4_test.txt"
FOLD4_EVALUATE = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/evaluation_setup/fold4_evaluate.txt"

development_metadata = pd.read_csv(DEVELOPMENT_METADATA, sep="\t", header=None)
development_metadata.columns = ["mp3_path", "scene"]

######### Fold1 ###########
fold1_train = pd.read_csv(FOLD1_TRAIN, sep="\t", header=None)
# Name the columns
fold1_train.columns = ["mp3_path", "scene"]
# Drop the scence column, since we have that information in the metadata frame
#fold1_train = development_metadata.drop(labels='scene', axis=1)

fold1_test = pd.read_csv(FOLD1_TEST, sep="\t", header=None)
# Name the columns
fold1_test.columns = ["mp3_path"]

fold1_evaluate = pd.read_csv(FOLD1_EVALUATE, sep="\t", header=None)
# Name the columns
fold1_evaluate.columns = ["mp3_path", "scene"]
####################################################

# Create One-Hot encoding
development_metadata = pd.get_dummies(development_metadata, columns=["scene"])
#development_metadata = development_metadata.drop(labels='mp3_path', axis=1)
#development_metadata_Y = list(development_metadata.to_numpy().astype("float32"))
#development_metadata_Y
#development_metadata
fold1_train

######## FOLD 1 #############
# Create X and y vectors for train data
fold1_train = pd.merge(fold1_train["mp3_path"], development_metadata, on="mp3_path", how="inner")
fold1_train_X = fold1_train["mp3_path"]
fold1_train_y = list(fold1_train.drop(labels="mp3_path", axis=1).to_numpy().astype("float32"))

# Create X and y vectors for test data
fold1_test = pd.merge(fold1_test["mp3_path"], development_metadata, on="mp3_path", how="inner")
fold1_test_X = fold1_test["mp3_path"]
fold1_test_y = list(fold1_test.drop(labels="mp3_path", axis=1).to_numpy().astype("float32"))

# Create X and y vectors for evluate data
fold1_evaluate = pd.merge(fold1_evaluate["mp3_path"], development_metadata, on="mp3_path", how="inner")
fold1_evaluate_X = fold1_evaluate["mp3_path"]
fold1_evaluate_y = list(fold1_evaluate.drop(labels="mp3_path", axis=1).to_numpy().astype("float32"))
#############################

## **Preprocess Audio**

In [68]:
###### PATH where the audio is stored (WITHOUT /audio!!!! since it is included in the mp3_path)
#AUDIO_PATH_DEVELOPMENT = "drive/My Drive/TU/Music/PartB/Data/Development/TUT-acoustic-scenes-2016-development/%s"
AUDIO_PATH_DEVELOPMENT = "data_e/TUT-acoustic-scenes-2016-development/%s"

def pytorch_preprocess(audio_file):

  # mel-spectrogram parameters
  SR = 12000
  N_FFT = 512
  N_MELS = 96
  HOP_LEN = 256

  #TODO without normalization?
  #src, sr = torchaudio.load(audio_file, normalization=True)
  src, sr = torchaudio.load(audio_file)
  # Resample? torchaudio.transforms.Resample(orig_freq=16000, new_freq=16000, resampling_method='sinc_interpolation')
  src = torchaudio.transforms.Resample(orig_freq=sr, new_freq=SR)(src)
  # Sch
  #src = src[0,:]
  

  #src = torch.Tensor(src)
  mel_specgram = transforms.MelSpectrogram(sample_rate=SR, n_mels=96, n_fft=N_FFT, hop_length=HOP_LEN)(src)  # (channel, n_mels, time)
  ret = torchaudio.transforms.AmplitudeToDB()(mel_specgram**2)
  #ret = torchaudio.transforms.AmplitudeToDB()(mel_specgram)
  ret = ret[np.newaxis, :]
  
  return ret

test_audio_str = (AUDIO_PATH_DEVELOPMENT % fold1_train_X[11])
src, sr = torchaudio.load(test_audio_str)
src.shape
pls = pytorch_preprocess(test_audio_str)
pls.shape

torch.Size([1, 2, 96, 1407])

## **Neural Network**

In [0]:
#### CNN
# transcription base class
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.batch_0 = nn.BatchNorm2d(2)
        self.conv1 = nn.Conv2d(2, 64, kernel_size=3)
        self.conv1_bn = nn.BatchNorm2d(64)
        self.conv1_max = nn.MaxPool2d((2,4))
        self.dropout1 = nn.Dropout2d(0.5)

        # Conv block 2
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv2_bn = nn.BatchNorm2d(128)
        self.conv2_max = nn.MaxPool2d((2,4))
        self.dropout2 = nn.Dropout2d(0.5)

        # Conv block 3
        self.conv3 = nn.Conv2d(128, 128, kernel_size=3)
        self.conv3_bn = nn.BatchNorm2d(128)
        self.conv3_max = nn.MaxPool2d((3,5))
        self.dropout3 = nn.Dropout2d(0.5)

        # Conv block 4
        self.conv4 = nn.Conv2d(128, 64, kernel_size=3)
        self.conv4_bn = nn.BatchNorm2d(64)
        self.conv4_max = nn.MaxPool2d((4,8))
        self.dropout4 = nn.Dropout2d(0.5)
        
        self.dense = nn.Linear(64, 15)

    def forward(self, x):
        #print("x",x.size())
        h1 = self.batch_0(x)
        # First Conv block
        h1 = self.dropout1(self.conv1_max(torch_func.relu(self.conv1_bn(self.conv1(h1)))))
        #print("h1",h1.size())
        # Second Conv block
        h1 = self.dropout2(self.conv2_max(torch_func.relu(self.conv2_bn(self.conv2(h1)))))
        #print("h1",h1.size())
        # Third Conv block
        h1 = self.dropout3(self.conv3_max(torch_func.relu(self.conv3_bn(self.conv3(h1)))))
        #print("h1",h1.size())
        # Fourth Conv block
        h1 = self.dropout4(self.conv4_max(torch_func.relu(self.conv4_bn(self.conv4(h1)))))
        #print("h1",h1.size())
        h1 = h1.view(-1, 64)
        #h1 = torch.flatten(h1)
        hn = self.dense(h1)
        #print("hn",hn.size())
        y = torch.sigmoid(hn)
        #print("y",y.size())
        return y

In [71]:
# use GPU for NN training?
g_use_cuda = True
# seed for RNG for reproducible results
seed = 12345

# helper class for arguments
class Args:
    pass

def train_nn(args, model, device, train_loader, optimizer, epoch):
    """
    Training loop for one epoch of NN training.
    Within one epoch, all the data is used once, we use mini-batch gradient descent.
    :param args: NN parameters for training and inference
    :param model: The model to be trained
    :param device: PyTorch device: CPU or GPU
    :param train_loader: Data provider
    :param optimizer: Optimizer (Gradient descent update algorithm)
    :param epoch: Current epoch number
    :return:
    """
    # set model to training mode (activate dropout layers for example).
    model.train()
    # we measure the needed time
    t = time.time()
    # iterate over training data
    for batch_idx, (data, target) in enumerate(train_loader):
      #print(pytorch_preprocess(DATA_PATH % data[0]).shape)
      #data = pytorch_preprocess(DATA_PATH % data[0]).view(-1, 96, 1336)
      try:
        #print("data",data)
        #print("target",target[0])
        data_tmp = []
        for d in data:
          #print("shape:", pytorch_preprocess(DATA_PATH % d).unsqueeze(0).shape)
          data_tmp.append(pytorch_preprocess(AUDIO_PATH_DEVELOPMENT % d))

        data = torch.cat(data_tmp)
        #target = torch.Tensor(target)
        #print("Target", target.shape)
        #print("data", data.shape)
        # move data to device (GPU) if necessary
        data, target = data.to(device), target.to(device)
        # reset optimizer
        optimizer.zero_grad()
        # forward pass (calculate output of network for input)
        output = model(data)
        #print("output", output.shape)
        # calculate loss
        loss = torch_func.mse_loss(output, target)
        # do a backward pass (calculate gradients using automatic differentiation and backpropagation)
        loss.backward()
        # udpate parameters of network using our optimizer
        optimizer.step()
        # print some outputs if we reached our logging intervall
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, took {:.2f}s'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), time.time()-t))
            t = time.time()
      except Exception as e:
        print(e)
        print("Could not read file:", data)

def test_nn(args, model, device, test_loader):
    """
    Function wich iterates over test data (eval or test set) and calculates loss.
    Here no parameter update is done
    :param args: NN parameters for training and inference
    :param model: The model to be tested
    :param device: PyTorch device: CPU or GPU
    :param test_loader: Data provider
    :return: cumulative test loss
    """
    # set model to inference mode (deactivate dropout layers for example).
    model.eval()
    # init cumulative loss
    test_loss = 0
    # do not calculate gradients since we do not want to do updates
    with torch.no_grad():
        # iterate over test data
        for data, target in test_loader:
          try:
            data_tmp = []
            for d in data:
              data_tmp.append(pytorch_preprocess(AUDIO_PATH_DEVELOPMENT % d))

            data = torch.cat(data_tmp)
            #target = torch.Tensor(target)
            # move data to device (GPU) if necessasry
            data, target = data.to(device), target.to(device)
            # forward pass (calculate output of network for input)
            output = model(data)
            # claculate loss and add it to our cumulative loss
            test_loss += torch_func.mse_loss(output, target, reduction='sum').item()  # sum up batch loss
          except Exception as e:
            print("Could not read file:", data)

    # output results of test run
    test_loss /= len(test_loader.dataset)
    print('Average loss: {:.4f}\n'.format(
        test_loss, len(test_loader.dataset)))
    
    return test_loss

def inference_cnn(args, model, device, data, outnum=50):
    """
    Function calculating the actual output of the network, given some input.
    :param args: NN parameters for training and inference
    :param model: The network to be used
    :param device: PyTorch device: CPU or GPU
    :param data: Data for which the output should be calculated
    :param outnum: Number of network outputs to reserve output memory
    :return: output of network
    """
    # set model to inference mode (deactivate dropout layers for example).
    model.eval()
    # reserve output memory
    in_shape = data.shape
    side = int((args.context-1)/2)
    outlen = in_shape[0] - 2*side
    output = np.zeros((outlen, outnum))
    # move input to device if necessary
    data = torch.from_numpy(data[None, None, :, :])
    data = data.to(device)
    # do not calculate gradients since we do not want to do updates
    with torch.no_grad():
        # iterate over input data
        for idx in range(outlen):
            # calculate output for input data
            output[idx, :] = model(data[:, :, idx:(idx+args.context), :])[0, :]
    # compensate for convolutional context and return output
    return np.vstack((np.zeros((side, outnum)), output, np.zeros((side, outnum))))
        


# parameters for NN training
name = 'CNN'
args = Args()
args.batch_size = 32
args.test_batch_size = 1000
args.max_epochs = 1
args.patience = 4
args.lr = 0.01
args.momentum = 0.5
args.no_cuda = not g_use_cuda
args.seed = 1
args.log_interval = 100
args.context = 25
args.step_size = 1

# setup pytorch
use_cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda:0" if use_cuda else "cpu")
print("device", device)

# create model and optimizer, we use plain SGD with momentum
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=args.lr)

# setup our datasets for training, evaluation and testing
kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {'num_workers': 4}
train_loader = torch.utils.data.DataLoader(list(zip(fold1_train_X.tolist(), fold1_train_y)),
                                            batch_size=args.batch_size, shuffle=True, **kwargs)
valid_loader = torch.utils.data.DataLoader(list(zip(fold1_evaluate_X.tolist(), fold1_evaluate_y)),
                                            batch_size=args.batch_size, shuffle=False, **kwargs)
test_loader = torch.utils.data.DataLoader(list(zip(fold1_test_X.tolist(), fold1_test_y)),
                                          batch_size=args.batch_size, shuffle=False, **kwargs)

# main training loop
best_test_loss = 9999
cur_patience = args.patience
for epoch in range(1, args.max_epochs + 1):
    # run one epoch of NN training
    train_nn(args, model, device, train_loader, optimizer, epoch)
    # validate on validation set
    print('\nValidation Set:')
    test_loss = test_nn(args, model, device, valid_loader)
    print("Test loss:", test_loss)
    # check for early stopping
    if test_loss < best_test_loss:
        torch.save(model.state_dict(), os.path.join("models", "cnn_model" + '.model'))
        best_test_loss = test_loss
        cur_patience = args.patience
    else:
        # if performance does not improve, we do not stop immediately but wait for 4 iterations (patience)
        if cur_patience <= 0:
            print('Early stopping, no improvement for %d epochs...' % args.patience)
            break
        else:
            print('No improvement, patience: %d' % cur_patience)
            cur_patience -= 1
    
# testing on test data
print('Evaluate CNN...')
print('Test Set:')
# calculate loss for test set
test_nn(args, model, device, test_loader)

device cuda:0
Train Epoch: 1 [0/880 (0%)]	Loss: 0.271152, took 53.96s

Validation Set:
Average loss: 0.9750

Test loss: 0.9749615628143836
Evaluate CNN...
Test Set:
Average loss: 0.9750



0.9749615628143836

In [0]:
# Evaluate
# Get all the outputs!
OUTPUTS = []
# Because of the files which cannot be read in.....
TEST_Y = []

with torch.no_grad():
  model.eval() ### Added eval new!
  for spec, annot in zip(fold1_test_X, fold1_test_y):
    try:
      test_feat = pytorch_preprocess(AUDIO_PATH_DEVELOPMENT % spec)
      test_feat = test_feat.to(device)
      out = model(test_feat)
      out = out.cpu()
      #t = Variable(torch.Tensor([0.05]))
      #out = (out>t).float() * 1

      out = out.numpy().reshape(15)
      OUTPUTS.append(out)
      TEST_Y.append(annot)
    except Exception as e:
      print(e)
      print("Could not read file:", spec)

In [80]:
# Get the accuracy
from sklearn.metrics import accuracy_score

accuracy_score(np.argmax(OUTPUTS, axis=1), np.argmax(TEST_Y, axis=1))

0.16206896551724137

In [35]:
#use_cuda = not args.no_cuda and torch.cuda.is_available()
#torch.manual_seed(seed)
device = torch.device("cuda:0")
print(device)
#torch.cuda.is_available()
print(torch.cuda.get_device_name(0))

cuda:0
Tesla T4
